## StarTracker WCS solutions - 28-Nov-22

Craig Lage - basically copying code from Merlin Fisher-Levine\
This version has the TAI correction.

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

import lsst.afw.image as afwImage
from lsst.summit.utils.blindSolving import runImchar, genericCameraHeaderToWcs, CommandLineSolver
# This requires the tickets/DM-31175 of summit_utils

In [ ]:
solver = CommandLineSolver(indexFiles='/scratch/mfisherlevine/astrometry_net/broiler.astrometry.net/~dstn/4100',
                           checkInParallel=True)

In [ ]:
cameras = [['Wide', 101], ['Narrow', 102]]
dates = [16, 22, 23, 24]
for [camera, num] in cameras:
    for date in dates:
        if camera == 'Narrow' and date in [15, 16, 22]:
            # No narrow camera data in early dates
            continue
        
        outfile = open(f"/scratch/cslage/starTracker/StarTracker_Summary_{camera}_202211{date}.txt", "w")
        outfile.write("SeqNum\tRA(header)\tDec(header)\tRA(Solve)\tDec(Solve)\tAz(hdr-fix)\tEl(header)\tAz(solve)\tEl(solve)\tDeltaAz\tDeltaEl\tRot\tRMS(asec)\n")
        path = f"/project/GenericCamera/{num}/2022/11/{date}/"
        for seqNum in range(200):
            try:    
                filename = path + f"GC{num}_O_202211{date}_{seqNum:06}.fits"
                exp = afwImage.ExposureF(filename)
                wcs = genericCameraHeaderToWcs(exp)
                exp.setWcs(wcs)
                results = runImchar(exp, 5, 25)
                if camera == 'Narrow':
                    # Need to loosen percentageScaleError because header SECPIX is wrong
                    solverResults = solver.run(exp, results.sourceCat,  percentageScaleError=60.0, silent=True)
                if camera == 'Wide':
                    solverResults = solver.run(exp, results.sourceCat, silent=True)
                newWcs = solverResults.wcs
                rms_error = solverResults.scatterArcseconds
                raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
                decSolve = newWcs.getSkyOrigin().getDec().asDegrees()

            except:
                print(f"{camera}, 202211{date}_{seqNum} Failed!")
                continue
            hdulist = pf.open(filename)
            header = hdulist[0].header
            raStart = header["RASTART"]
            decStart = header["DECSTART"]
            azStart = header["AZSTART"]
            azStart = 360.0 - azStart # Compensating for Az sign error
            if azStart > 360.0:
                azStart -= 360.0
            if azStart < 0.0:
                azStart += 360.0

            elStart = header["ELSTART"]
            rotpa = header["ROTPA"]
            rot = newWcs.getRelativeRotationToWcs(wcs).asDegrees()
            time = Time(header["DATE-OBS"], scale='tai')
            altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
            skyLocation = SkyCoord(raSolve * u.deg, decSolve * u.deg)
            obsAltAz = skyLocation.transform_to(altAz)
            az = obsAltAz.az.deg
            el = obsAltAz.alt.deg
            deltaAz = azStart - az
            if deltaAz > 360.0:
                deltaAz -= 360.0
            if deltaAz < 0.0:
                deltaAz += 360.0
            deltaEl = elStart - el
            print(f"{camera}, 202211{date}_{seqNum}, {deltaAz:.5f}, {deltaEl:.5f}")
            outfile.write(f"{seqNum}\t{raStart:.5f}\t{decStart:.5f}\t{raSolve:.5f}\t{decSolve:.5f}\t{azStart:.5f}\t{elStart:.5f}\t{az:.5f}\t{el:.5f}\t{deltaAz:.5f}\t{deltaEl:.5f}\t{rot:.5f}\t{rms_error:.5f}\n")
        outfile.close()
    